\begin{equation*}
P_t = E_{t}\left[ \frac{u(d_{t+1})}{u(d_t)} (P_{t+1} + d_{t+1}) \right]
\end{equation*}

In [ ]:
class LucasEconomy:
    
    def __init__(CRRA, DiscFac, DivDist):
        
        self.CRRA = CRRA
        self.DiscFac = DiscFac
        self.DivDist = DivDist